### 离散数值迭代的非线性带不等式约束的LQR解法

在实际应用中，非线性系统的LQR问题通常需要通过数值迭代的方法求解，尤其是当系统受到不等式约束时。以下是基于前向-后向迭代的数值解法的详细描述：

#### 问题描述
我们需要求解以下优化问题：
$$
\min \quad J = \sum_{k=0}^{N-1} L(x_k, u_k, t_k) + \Phi(x_N),
$$
约束条件为：
1. **动态约束**：
$$
x_{k+1} = f(x_k, u_k, t_k),
$$
2. **不等式约束**：
$$
g(x_k, u_k, t_k) \leq 0.
$$

#### 哈密顿函数构造
为了处理上述优化问题，我们引入协态变量 $\lambda_k$ 和拉格朗日乘子 $\mu_k$，构造离散形式的哈密顿函数：
$$
H(x_k, u_k, \lambda_{k+1}, \mu_k, t_k) = L(x_k, u_k, t_k) + \lambda_{k+1}^T f(x_k, u_k, t_k) + \mu_k^T g(x_k, u_k, t_k),
$$
其中 $\mu_k \geq 0$ 是与不等式约束相关的乘子。

#### 数值迭代方法
数值解法基于前向-后向迭代，具体步骤如下：

1. **初始化**：
   - 设置初始状态 $x_0$。
   - 初始化控制序列 $u_k$（例如，零控制或随机初始化）。
   - 初始化拉格朗日乘子 $\mu_k$（如果有不等式约束）。

2. **前向积分**：
   - 从 $x_0$ 出发，利用当前的 $u_k$，通过动态方程计算状态序列 $x_k$：
     $$
     x_{k+1} = x_k + \Delta t \cdot f(x_k, u_k, t_k).
     $$

3. **后向积分**：
   - 从终端条件 $\lambda_N = \frac{\partial \Phi}{\partial x_N}$ 出发，利用当前的 $x_k$ 和 $u_k$，通过协态方程计算协态序列 $\lambda_k$：
     $$
     \lambda_k = \lambda_{k+1} + \Delta t \cdot \left( \frac{\partial L}{\partial x_k} + \lambda_{k+1}^T \frac{\partial f}{\partial x_k} + \mu_k^T \frac{\partial g}{\partial x_k} \right).
     $$
     注意，这里需要使用前向积分得到的 $x_k$ 和 $u_k$。

4. **更新控制变量**：
   - 对每个时间步 $k$，根据控制最优性条件更新 $u_k$：
     $$
     \frac{\partial H}{\partial u_k} = \frac{\partial L}{\partial u_k} + \lambda_{k+1}^T \frac{\partial f}{\partial u_k} + \mu_k^T \frac{\partial g}{\partial u_k} = 0.
     $$
   - 如果 $\frac{\partial H}{\partial u_k} = 0$ 是线性方程，可以直接求解 $u_k$。
   - 如果是非线性方程，使用梯度下降法或牛顿法迭代求解：
     $$
     u_k^{(i+1)} = u_k^{(i)} - \eta \cdot \frac{\partial H}{\partial u_k}.
     $$
   - 如果有不等式约束 $g(x_k, u_k, t_k) \leq 0$，将 $u_k$ 投影到可行域内：
     $$
     u_k = \arg\min_{u} \| u - u_k \|^2, \quad \text{subject to } g(x_k, u, t_k) \leq 0.
     $$

5. **更新拉格朗日乘子**：
   - 在算完当前步的最优控制量 $u_k^*$ 之后，使用当前 $x_k$ 一起更新乘子。
   - 根据互补条件更新 $\mu_k$：
     $$
     \mu_k = \max(0, \mu_k + \alpha \cdot g(x_k, u_k, t_k)).
     $$

6. **收敛判定**：
   - 定义收敛准则，例如：
     $$
     \| u_k^{(i+1)} - u_k^{(i)} \| < \epsilon.
     $$
   - 这一步需要在当前循环结束后，对所有时间步都检查是否满足。
   - 如果满足收敛条件，则停止迭代；否则，返回前向积分步骤。

#### 总结
上述方法通过前向-后向迭代，逐步逼近最优解。每次迭代中，状态变量 $x_k$、控制变量 $u_k$ 和协态变量 $\lambda_k$ 都会被更新。不等式约束通过投影法或拉格朗日乘子法处理，确保解的可行性。

## 1. iLQR：轨迹优化示例（车辆模型，C++ 实现）

---

### 1.1 变量与动力学模型
- 状态：$x=[p_x, p_y, \theta, v]^\top$。
- 控制：$u=[a,\omega]^\top$。

连续动力学为：
$$
\begin{aligned}
\dot p_x &= v\cos\theta,\\
\dot p_y &= v\sin\theta,\\
\dot\theta &= \omega,\\
\dot v &= a.
\end{aligned}
$$

离散化（显式 Euler，步长 $d t$）：
$$
x_{k+1} = x_k + d t\, f(x_k,u_k),\qquad f(x,u)=\begin{bmatrix}v\cos\theta\\v\sin\theta\\\omega\\a\end{bmatrix}.
$$

---

### 1.2 代价函数（stage + terminal）
阶段代价与终端代价为：
$$
\ell_k(x_k,u_k)=(x_k-x^{\mathrm{ref}}_k)^\top Q (x_k-x^{\mathrm{ref}}_k)+u_k^\top R u_k,
$$
$$
\ell_N(x_N)=(x_N-x^{\mathrm{ref}}_N)^\top Q_f (x_N-x^{\mathrm{ref}}_N).
$$
总代价：
$$
J(X,U)=\sum_{k=0}^{N-1}\ell_k(x_k,u_k)+\ell_N(x_N).
$$
（示例权重：$Q=\mathrm{diag}(100,100,10,1)$，$R=\mathrm{diag}(1,0.1)$，$Q_f$ 放大以强调终端精度。）


---

### 1.3 线性化与二次展开
在当前轨迹 $(x_k,u_k)$ 处，动力学和代价值做一阶/二阶展开：

动力学线性化（增量形式）：
$$
d x_{k+1} \approx A_k\,d x_k + B_k\,d u_k,
$$
其中
$$
A_k=\frac{\partial x_{k+1}}{\partial x}\Big|_{x_k,u_k},\qquad B_k=\frac{\partial x_{k+1}}{\partial u}\Big|_{x_k,u_k}.
$$
（显式 Euler 下 $A_k=I+d t\,\frac{\partial f}{\partial x}$，$B_k=dt \frac{\partial f}{\partial u}$。）

代价二次近似：
$$
\ell_k(x_k+d x,u_k+d u) \approx \ell_k + \ell_x^\top d x + \ell_u^\top d u + \tfrac12 \begin{bmatrix}d x\\d u\end{bmatrix}^\top \begin{bmatrix}\ell_{xx} & \ell_{xu}\\\ell_{ux} & \ell_{uu}\end{bmatrix} \begin{bmatrix}d x\\d u\end{bmatrix}.
$$


### 1.3.1 Hfull 切分与各子块含义（补充说明）
- 先定义：$z=\begin{bmatrix}x\\u\end{bmatrix}\in\mathbb{R}^{n_x+n_u}$，因此对 $l(x,u)$ 关于 $z$ 的 Hessian（`Hfull`）的维度为 $(n_x+n_u)\times(n_x+n_u)$。


- Hessian 的块结构（按行列分块）为：
$$
H_{full} = \begin{bmatrix} H_{xx} & H_{xu} \\
H_{ux} & H_{uu} \end{bmatrix}
$$
其中
  - $H_{xx}=\frac{\partial^2 l}{\partial x^2}$，形状 $n_x\times n_x$；
  - $H_{xu}=\frac{\partial^2 l}{\partial x\partial u}$，形状 $n_x\times n_u$；
  - $H_{ux}=\frac{\partial^2 l}{\partial u\partial x}$，形状 $n_u\times n_x$；
  - $H_{uu}=\frac{\partial^2 l}{\partial u^2}$，形状 $n_u\times n_u$。


### 1.4.x 状态值 / 动作值 函数与 Q 展开（详细推导）

- **贝尔曼原理（离散）**：定义最优值函数（状态值函数）

$$
V_k(x_k) = \min_{u_k}\;\big[\,\ell_k(x_k,u_k) + V_{k+1}(x_{k+1})\,\big],\quad x_{k+1}=f(x_k,u_k).
$$

动作值函数（Action-value）定义为在给定 $(x_k,u_k)$ 下的局部值：

$$
Q_k(x_k,u_k) = \ell_k(x_k,u_k) + V_{k+1}\big(f(x_k,u_k)\big).
$$

- **二阶展开（在标称轨迹点）**：设标称点 $(x_k,u_k)$，令增量 $d x,d u$，对 $\ell$ 与 $V_{k+1}$ 做二阶展开，并用线性化动力学

$$
d x_{k+1}\approx A\,d x + B\,d u,\quad A=\partial x_{k+1}/\partial x,\;B=\partial x_{k+1}/\partial u.
$$

将展开代入 $Q$ 得到局部二次近似：

$$
Q(x+dx,u+du) \approx Q(x_k,u_k) + Q_x^Td x + Q_u^Td u + \tfrac12[d x;d u]^T \begin{bmatrix}Q_{xx} & Q_{xu} \\
Q_{ux} & Q_{uu}\end{bmatrix} [d x;d u].
$$

其中各项由 $\ell$、$A,B$ 与后验值函数导数构成（记 $V_{k+1,x} = V_{k+1,x}$，$V_{k+1,xx}=V_{k+1,xx}$）：

$$
\begin{aligned}
Q_x &= \ell_x + A^T V_{k+1,x},\\
Q_u &= \ell_u + B^T V_{k+1,x},\\
Q_{xx} &= \ell_{xx} + A^T V_{k+1,xx} A,\\
Q_{uu} &= \ell_{uu} + B^T V_{k+1,xx} B,\\
Q_{ux} &= \ell_{ux} + B^T V_{k+1,xx} A.\qquad(\text{注意 }Q_{ux} \in \mathbb{R}^{n_u \times n_x})
\end{aligned}
$$

(在代码中常按维度将 $Q_{ux}$ 视为 $n_u \times n_x$，而 $Q_{xu}=Q_{ux}^\top$。)

- **求取最优增量控制**（二次子问题的解析解，带 Quu 正则）：

$$
k = -\,\mathrm{inv}(Q_{uu}+\mu I)\;Q_u,
\qquad
K = -\,\mathrm{inv}(Q_{uu}+\mu I)\;Q_{ux}.
$$

其中 $k$ 为前馈（feedforward），$K$ 为反馈增益，$\mu$ 为 Levenberg-Marquardt 风格的正则化项以保证数值稳定。

- **价值函数更新（后向传递）**：把最优控制代入得到新的状态值的一阶/二阶导数（常用、且经化简的形式）：

$$
V_x = Q_x - Q_{ux}^T\,Q_{uu}^{-1}\,Q_u,
\qquad
V_{xx} = Q_{xx} - Q_{ux}^T\,Q_{uu}^{-1}\,Q_{ux}.
$$

并且常数项的变化（用于累计总代价下降）为

$$
d V = -\tfrac12\,Q_u^T\,Q_{uu}^{-1}\,Q_u.
$$

（数值实现中应避免显式求逆，而用线性求解器或矩阵分解来求 $Q_{uu}^{-1}Q_u$ / $Q_{uu}^{-1}Q_{ux}$。）


#### 1.4.x1 详细推导：从 $\ell$ 与 $V_{k+1}$ 到 $Q$ 的各项（逐步代数）

1) 先把阶段代价与后验值做二阶展开（在标称点取增量 $d x$, $d u$）：

$$
\begin{aligned}
\ell(x+d x, u+d u) &\approx \ell + \ell_x^Td x + \ell_u^Td u + \tfrac12 [d x \quad d u]^T \begin{bmatrix}\ell_{xx} & \ell_{xu} \\
\ell_{ux} & \ell_{uu} \end{bmatrix} [d x & d u], \\
V_{k+1}(x_{k+1}+d x_{k+1}) &\approx V_{k+1} + V_{k+1,x}^Td x_{k+1} + \tfrac12\,d x_{k+1}^T V_{k+1,xx}\,d x_{k+1}.
\end{aligned}
$$

2) 用线性化动力学把 $d x_{k+1}$ 用 $d x_k$, $d u_k$ 表示：

$$d x_{k+1} \approx A\,d x_k + B\,d u_k.$$

3) 把展开代入动作值函数定义（在给定 $(x_k,u_k)$ 下）为
$$
Q_k(x_k,u_k) = \ell_k(x_k,u_k) + V_{k+1}\big(f(x_k,u_k)\big),
$$
收集一次/二次项得：
$$
Q_x = \ell_x + A^T V_{k+1,x},\qquad Q_u = \ell_u + B^T V_{k+1,x}.
$$

- 二次项（块矩阵）
$$
Q_{xx} = \ell_{xx} + A^T V_{k+1,xx} A,\quad Q_{uu} = \ell_{uu} + B^T V_{k+1,xx} B,\quad Q_{ux} = \ell_{ux} + B^T V_{k+1,xx} A.
$$

（说明：在代码实现中通常把 $Q_{ux}$ 存为 $n_u\times n_x$，而 $Q_{xu}=Q_{ux}^\top$。）

4) 对 $d u$ 求极小化：令

$$
\frac{\partial Q}{\partial d u} = 0
$$

得

$$Q_u + Q_{uu}\,d u + Q_{ux}\,d x = 0 \quad\Rightarrow\quad d u^*(d x)= -Q_{uu}^{-1}\big(Q_u + Q_{ux}\,d x\big).$$

写成前馈/反馈形式：
$$d u^* = k + K\,d x,\qquad k = -Q_{uu}^{-1}Q_u,\quad K = -Q_{uu}^{-1}Q_{ux}.$$

1) 把最优 $d u^*$ 代回，得到状态值函数的一阶二阶导数更新（代数化简后常用的紧凑形式）：

$$
V_x = Q_x - Q_{ux}^T Q_{uu}^{-1} Q_u,\qquad V_{xx} = Q_{xx} - Q_{ux}^T Q_{uu}^{-1} Q_{ux}.
$$
常数项的改变量（用于评估代价下降）为
$$d V = -\tfrac12\,Q_u^T Q_{uu}^{-1} Q_u.
$$
（推导提示：代入 $d u^*$ 并对 $d x$ 求导与展开即可得到上面结果；完整逐项代回可在纸上按矩阵代数验证，步骤直接照上述代入展开即可。）


1) 数值注意事项：
- 用 $solve$ 或矩阵分解替代显式求逆；对小维度（如 $n_u\le 2$）可用闭式解并做正则化；始终保持 $V_{k+1,xx}$ 对称（对称化 $(V_{k+1,xx}+V_{k+1,xx}^\top)/2$）。
- 若 $Q_{uu}$ 非 PD，则增大正则 $\mu$ 直到可解。
- 
---


### 1.4.x2 逐项代入与代数化简（完整步骤，补充 $V$ 的变化与关系）

下面把推导写成逐项代数展开的形式，补充 $V$（状态值函数）的变化，并明确常数项、一次项、二次项的关系。

1) **记局部二次近似（以增量 $d x$, $d u$ 表达）**：

$$
Q(x + d x, u + d u)=\underbrace{c}_{\text{常数}} + Q_x^Td x + Q_u^Td u + \tfrac12d x^T Q_{xx} d x + d x^T Q_{xu} d u + \tfrac12 d u^T Q_{uu} d u.
$$

其中：
- $c$ 是常数项，表示 $Q$ 在当前点的值；
- $Q_x$, $Q_u$ 是一次项系数，分别对应 $d x$, $d u$ 的线性变化；
- $Q_{xx}$, $Q_{uu}$, $Q_{xu}$ 是二次项系数，描述 $d x$, $d u$ 的二次变化关系。

> **注释 1：** $Q(x + d x, u + d u)$ 是 $Q$ 的局部二次近似形式，描述了 $Q$ 在当前点附近的变化情况，因此可以理解为 $d Q$ 的展开形式。

> **注释 2：** 常数项 $c$ 是 $Q$ 在当前点 $(x, u)$ 的值，即 $c = Q(x, u)$。它来源于 $Q$ 的定义，通常由代价函数 $l(x, u)$ 和终端代价 $V_f(x)$ 共同决定：
> $$
> Q(x, u) = l(x, u) + V_f(f(x, u)),
> $$
> 其中 $f(x, u)$ 是系统的动态模型，$l(x, u)$ 是即时代价，$V_f(x)$ 是终端代价函数。

> **注释 3：** 根据泰勒展开公式，$Q(x + d x, u + d u)$ 可以展开为：
> $$
> Q(x + d x, u + d u) = Q(x, u) + \frac{\partial Q}{\partial x}d x + \frac{\partial Q}{\partial u}d u + \frac{1}{2}d x^T \frac{\partial^2 Q}{\partial x^2}d x + d x^T \frac{\partial^2 Q}{\partial x \partial u}d u + \frac{1}{2}d u^T \frac{\partial^2 Q}{\partial u^2}d u.
> $$
> 这正是 $Q(x + d x, u + d u)$ 的具体形式，其中各项系数对应 $Q_x$, $Q_u$, $Q_{xx}$, $Q_{xu}$ 和 $Q_{uu}$。

> **注释 4：** 为了简化表达，通常将 $Q(x + d x, u + d u)$ 写作 $Q(d x, d u)$，省略了对基点 $(x, u)$ 的显式依赖，但实际含义仍然是围绕 $(x, u)$ 的泰勒展开。

2) **$V^{\text{new}}$ 的表达式**：

$V^{\text{new}}$ 表示新的状态值函数，定义为 $Q$ 在最优 $u$ 下的最小值：
$$
V^{\text{new}}(x + d x) = \min_{d u} Q(x + d x, u + d u).
$$
将 $d u^* = k + Kd x$ 带入 $Q(x + d x, u + d u)$，得到：
$$
V^{\text{new}}(x + d x) = c + Q_u^T k + \tfrac{1}{2} k^T Q_{uu} k + \big(Q_x + Q_{xu}^T k + Q_u^T K\big)^T d x + \tfrac{1}{2} d x^T \big(Q_{xx} + K^T Q_{uu} K + Q_{xu} K + K^T Q_{xu}^T\big) d x.
$$

3) **$V_x^{\text{new}}$ 的推导**：

$V_x^{\text{new}}$ 表示 $V^{\text{new}}$ 对状态 $x$ 的梯度，由 $V^{\text{new}}$ 的线性项系数给出：
$$
V_x^{\text{new}} = Q_x - Q_{ux}^T Q_{uu}^{-1} Q_u.
$$

4) **$V_{xx}^{\text{new}}$ 的推导**：

$V_{xx}^{\text{new}}$ 表示 $V^{\text{new}}$ 对状态 $x$ 的二阶导数，由 $V^{\text{new}}$ 的二次项系数给出：
$$
V_{xx}^{\text{new}} = Q_{xx} - Q_{ux}^T Q_{uu}^{-1} Q_{ux}.
$$

5) **总结更新公式**：
$$
\boxed{
V_x^{\text{new}} = Q_x - Q_{ux}^T Q_{uu}^{-1} Q_u,\quad
V_{xx}^{\text{new}} = Q_{xx} - Q_{ux}^T Q_{uu}^{-1} Q_{ux},\quad
d V = -\tfrac12 Q_u^T Q_{uu}^{-1} Q_u.
}
$$

---

如果需要进一步补充推导细节或添加数值验证示例，请告诉我！


### 1.4.x4 反向传播与前向传播的详细推导过程

为了更清晰地理解反向传播和前向传播的过程，以下给出详细的推导步骤，并说明每一步需要记录哪些信息以便前向传播时使用：

#### **反向传播（Backward Pass）**

1. **初始条件（终端时刻 $N$）**：
   - 已知终端状态值函数 $V_N(x)$：
     - 常数项：$C_N$
     - 一阶项：$V_{N,x}$
     - 二阶项：$V_{N,xx}$
   - 即时代价函数 $l(x, u)$ 和系统动态 $f(x, u)$。

2. **从终端时刻 $N$ 反向递推到 $N-1$**：
   - 计算 $Q_{N-1}$ 函数：
     $$
     Q_{N-1}(x, u) = l(x, u) + V_N(f(x, u)).
     $$
     - 对 $l(x, u)$ 进行二次展开：
       $$
       l(x, u) \approx c_l + l_x^T d x + l_u^T d u + \tfrac{1}{2} d x^T l_{xx} d x + d x^T l_{xu} d u + \tfrac{1}{2} d u^T l_{uu} d u.
       $$
     - 对 $f(x, u)$ 进行一阶泰勒展开：
       $$
       f(x, u) \approx f(x_0, u_0) + f_x d x + f_u d u.
       $$
     - 将 $V_N(f(x, u))$ 展开：
       $$
       V_N(f(x, u)) \approx C_N + V_{N,x}^T f(x, u) + \tfrac{1}{2} f(x, u)^T V_{N,xx} f(x, u).
       $$
     - 合并得到 $Q_{N-1}$ 的局部二次近似：
       $$
       Q_{N-1}(d x, d u) = c + Q_x^T d x + Q_u^T d u + \tfrac{1}{2} d x^T Q_{xx} d x + d x^T Q_{xu} d u + \tfrac{1}{2} d u^T Q_{uu} d u.
       $$

   - **$Q$ 的各阶导数表达式**：
     - 一阶导数：
       $$
       Q_x = l_x + f_x^T V_{N,x}, \quad Q_u = l_u + f_u^T V_{N,x}.
       $$
     - 二阶导数：
       $$
       Q_{xx} = l_{xx} + f_x^T V_{N,xx} f_x, \quad Q_{uu} = l_{uu} + f_u^T V_{N,xx} f_u, \quad Q_{xu} = l_{xu} + f_x^T V_{N,xx} f_u.
       $$

   - 求解最优 $d u_{N-1}^*$：
     $$
     d u_{N-1}^* = k_{N-1} + K_{N-1} d x, \quad k_{N-1} = -Q_{uu}^{-1} Q_u, \quad K_{N-1} = -Q_{uu}^{-1} Q_{ux}.
     $$
     - **记录信息**：
       - 控制增量策略 $k_{N-1}$ 和 $K_{N-1}$。

   - 更新 $V_{N-1}$：
     $$
     \begin{aligned}
     C_{N-1} &= C_N - \tfrac{1}{2} Q_u^T Q_{uu}^{-1} Q_u=C_N+\tfrac{1}{2} Q_u^Tk_{N-1}, \\
     V_{N-1,x} &= Q_x - Q_{ux}^T Q_{uu}^{-1} Q_u=Q_x+Q_{ux}^Tk_{N-1}, \\
     V_{N-1,xx} &= Q_{xx} - Q_{ux}^T Q_{uu}^{-1} Q_{ux}=Q_{xx}+Q_{ux}^TK_{N-1}.
     \end{aligned}
     $$
     - **记录信息**：
       - 更新后的值函数项 $C_{N-1}$、$V_{N-1,x}$ 和 $V_{N-1,xx}$。

3. **重复递推**：
   - 按上述步骤从 $N-1$ 递推到 $0$，依次计算 $V_{k,x}$ 和 $V_{k,xx}$。
   - **记录信息**：
     - 每个时刻的控制策略 $k_k$ 和 $K_k$。
     - 每个时刻的值函数项 $C_k$、$V_{k,x}$ 和 $V_{k,xx}$。

#### **前向传播（Forward Pass）**

1. **初始条件（起始时刻 $0$）**：
   - 已知初始状态 $x_0$。
   - 初始状态偏差 $d x_0$：
     - 通常由外部扰动或初始状态偏差引起，例如 $d x_0 = x_0 - x_0^{\text{ref}}$，其中 $x_0^{\text{ref}}$ 是参考初始状态。

2. **从 $x_0$ 前向传播到 $x_N$**：
   - 计算控制增量：
     $$
     d u_k = k_k + K_k d x_k.
     $$
     - **需要的信息**：
       - 从反向传播中记录的 $k_k$ 和 $K_k$。
   - 更新状态：
     $$
     x_{k+1} = f(x_k, u_k).
     $$
     - **需要的信息**：
       - 系统动态 $f(x, u)$。

3. **重复传播**：
   - 按上述步骤从 $x_0$ 前向传播到 $x_N$，得到完整的状态和控制序列。
   - **记录信息**：
     - 每个时刻的状态 $x_k$ 和控制 $u_k$。

#### **总结**

- **反向传播**：从终端时刻 $N$ 开始，利用 $V_N(x)$ 和 $l(x, u)$ 递推计算 $V_{k,x}$ 和 $V_{k,xx}$，并记录控制策略 $k_k$ 和 $K_k$。
- **前向传播**：从初始状态 $x_0$ 开始，利用反向传播得到的 $k_k$ 和 $K_k$ 计算状态和控制序列。

通过以上详细推导，可以清晰地理解反向传播和前向传播的过程。如果需要进一步补充或修改，请告诉我！


### iLQR 正则化公式及其影响

正则化前：
$$d u^* = k + K\,d x,\qquad k = -Q_{uu}^{-1}Q_u,\quad K = -Q_{uu}^{-1}Q_{ux}.$$

$$
\begin{aligned}
C_{N-1} &= C_N - \tfrac{1}{2} Q_u^T Q_{uu}^{-1} Q_u=C_N+\tfrac{1}{2} Q_u^Tk_{N-1}, \\
V_{N-1,x} &= Q_x - Q_{ux}^T Q_{uu}^{-1} Q_u=Q_x+Q_{ux}^Tk_{N-1}, \\
V_{N-1,xx} &= Q_{xx} - Q_{ux}^T Q_{uu}^{-1} Q_{ux}=Q_{xx}+Q_{ux}^TK_{N-1}.
\end{aligned}
$$

在 iLQR 算法中，为了提高数值稳定性，通常对 $Q_{uu}$ 矩阵进行正则化。正则化的公式如下：

$$
Q_{uu}^{\text{reg}} = Q_{uu} + \mu I
$$

其中：
- $Q_{uu}$ 是原始的 Hessian 矩阵。
- $\mu$ 是正则化系数，通常是一个小的正数。
- $I$ 是单位矩阵。


正则化的作用是确保 $Q_{uu}$ 是正定的，从而避免在计算控制增量时出现数值问题。

在 iLQR 算法中，正则化后的 $Q_{uu}^{\text{reg}}$ 被带入相关公式后，可以展开如下：

#### 正则化后对 $V_x$, $V_{xx}$ 和 $d V$ 的影响

$$
K =- \left(Q_{uu} + \mu I\right)^{-1} Q_{ux}
$$
$$
k =- \left(Q_{uu} + \mu I\right)^{-1} Q_{u}
$$
1. **$V_x$ 的变化**：
   $$
   V_x = Q_x + Q_{ux}^T k
$$
$$
   V_x = Q_x - Q_{ux}^T\left[\left(Q_{uu} + \mu I\right)^{-1} Q_{u}\right] 
   $$
2. **$V_{xx}$ 的变化**：
   $$
   V_{xx} = Q_{xx} + Q_{ux}^T K
   $$
   同样，由于 $K$ 的变化，$V_{xx}$ 也会受到正则化的影响。
   $$
   V_{xx} = Q_{xx} - Q_{ux}^T \left[\left(Q_{uu} + \mu I\right)^{-1} Q_{ux}\right] 
   $$
   
3. **$d V$ 的变化**：
   $$
   d V = \frac{1}{2} d u^T Q_{uu}^{\text{reg}} d u + d u^T Q_u
   $$
   正则化会直接影响 $Q_{uu}^{\text{reg}}$，从而改变 $d V$ 的值。
   $$
   d V = \frac{1}{2} d u^T \left(Q_{uu} + \mu I\right) d u + d u^T Q_u
   $$

#### 正则化对 $d u$ 的影响

带入$Q_{uu}^{\text{reg}}$控制增量 $d u^*$ 的计算公式为：
$$
d u^*= -\left(Q_{uu} + \mu I\right)^{-1}Q_u-\left(Q_{uu} + \mu I\right)^{-1} Q_{ux}dx
$$
其中：
- $\left(Q_{uu} + \mu I\right)^{-1}$ 的引入确保了 $Q_{uu}$ 的正定性，从而避免了数值不稳定问题。

通过选择合适的正则化系数 $\mu$，可以在数值稳定性和优化精度之间取得平衡。